# Frame Unwrapping

## Context

At time-of-flight neutron sources recording event-mode, time-stamps of detected neutrons are written to files in an `NXevent_data` group.
This contains two main time components, `event_time_zero` and `event_time_offset`.
The sum of the two would typically yield the absolute detection time of the neutron.
For computation of wavelengths or energies during data-reduction, a time-of-flight is required.
In principle the time-of-flight could be equivalent to `event_time_offset`, and the emission time of the neutron to `event_time_zero`.
Since an actual computation of time-of-flight would require knowledge about chopper settings, detector positions, and whether the scattering of the sample is elastic or inelastic, this may however not be the case in practice.
Instead, the data acquisition system may, e.g., record the time at which the proton pulse hits the target as `event_time_zero`, with `event_time_offset` representing the offset since then.

We refer to the process of "unwrapping" these time stamps into an actual time-of-flight as *frame unwrapping*, since `event_time_offset` "wraps around" with the period of the proton pulse and neutrons created by different proton pulses may be recorded with the *same* `event_time_zero`.
The figures in the remainder of this document will clarify this.

In [ ]:
import plopp as pp
import scipp as sc
import sciline as sl
from scippneutron.chopper import DiskChopper
from scippneutron.tof import unwrap
import tof as tof_pkg

Hz = sc.Unit("Hz")
deg = sc.Unit("deg")
meter = sc.Unit("m")

## Default mode

Often there is a 1:1 correspondence between source pulses and neutron pulses propagated to the sample and detectors.

In this first example:

- We begin by creating a source of neutrons which mimics the ESS source.
- We set up a single chopper with a single opening
- We place 4 'monitors' along the path of the neutrons (none of which absorb any neutrons)

In [ ]:
source = tof_pkg.Source(facility="ess", pulses=5)
chopper = tof_pkg.Chopper(
    frequency=14.0 * Hz,
    open=sc.array(dims=["cutout"], values=[0.0], unit="deg"),
    close=sc.array(dims=["cutout"], values=[3.0], unit="deg"),
    phase=85.0 * deg,
    distance=8.0 * meter,
    name="chopper",
)
detectors = [
    tof_pkg.Detector(distance=20.0 * meter, name="beam"),
    tof_pkg.Detector(distance=60.0 * meter, name="sample"),
    tof_pkg.Detector(distance=80.0 * meter, name="monitor"),
    tof_pkg.Detector(distance=108.0 * meter, name="detector"),
]

model = tof_pkg.Model(source=source, choppers=[chopper], detectors=detectors)
results = model.run()
pl = results.plot(cmap="viridis_r")

for i in range(source.pulses):
    pl.ax.axvline(
        i * (1.0 / source.frequency).to(unit="us").value, color="k", ls="dotted"
    )
    x = [
        results[det.name].toas.data["visible"][f"pulse:{i}"].coords["toa"].min().value
        for det in detectors
    ]
    y = [det.distance.value for det in detectors]
    pl.ax.plot(x, y, "--o", color="magenta", lw=3)
    if i == 0:
        pl.ax.text(
            x[2], y[2] * 1.05, "pivot time", va="bottom", ha="right", color="magenta"
        )

In the figure above, the dotted vertical lines represent the `event_time_zero` of each pulse,
i.e. the start of a new origin for `event_time_offset` recorded at the various detectors.

The span between two dotted lines is called a 'frame'.

The figure gives a good representation of the situation at each detector:

- **beam** monitor: all the arrival times at the detector are inside the same frame within which the neutrons were created.
- **sample**: all the arrival times are offset by one frame
- **monitor**: most of the neutrons arrive with an offset of two frames, but a small amount of neutrons (shortest wavelengths) only have a 1-frame offset
- **detector**: most of the neutrons arrive with an offset of two frames, but a small amount of neutrons (longest wavelengths) have a 3-frame offset

We can further illustrate this by making histograms of the `event_time_offset` of the neutrons for each detector:

In [ ]:
subplots = pp.tiled(2, 2, figsize=(9, 6))
nxevent_data = results.to_nxevent_data()
for i, det in enumerate(detectors):
    data = nxevent_data["detector_number", i]
    subplots[i // 2, i % 2] = (
        data.bins.concat()
        .hist(event_time_offset=200)
        .plot(title=f"{det.name}={det.distance:c}", color=f"C{i}")
    )
    f = subplots[i // 2, i % 2]
    xpiv = min(
        da.coords["toa"].min() % (1.0 / source.frequency).to(unit="us")
        for da in results[det.name].toas.data["visible"].values()
    ).value
    f.ax.axvline(xpiv, ls="dashed", color="magenta", lw=2)
    f.ax.text(xpiv, 20, "pivot time", rotation=90, color="magenta")
    f.canvas.draw()
subplots

### Pivot time

To compute the time-of-flight for a neutron, we need to identify which source pulse it originated from.

In the first figure, the pink lines represent the earliest recorded arrival time at each detector:
we know that within a given frame at a selected detector,
any neutron recorded at a time earlier than this 'pivot' time must from from a previous pulse.

The position of the pink lines is repeated in the second figure (above).
We can use this knowledge to unwrap the frames and compute the absolute time-of-arrival of the neutrons at the detectors.

### Computing time-of-flight

The pivot time and the resulting offsets can be computed from the properties of the source pulse and the chopper cascade.

We describe in this section the workflow that computes time-of-flight,
given `event_time_zero` and `event_time_offset` for neutron events,
as well as the properties of the source pulse and the choppers in the beamline.

The workflow can be visualized as follows:

In [ ]:
workflow = sl.Pipeline(unwrap.providers(), params=unwrap.params())

workflow[unwrap.Facility] = "ess"
workflow[unwrap.Choppers] = {
    "chopper": DiskChopper(
        frequency=-chopper.frequency,
        beam_position=sc.scalar(0.0, unit="deg"),
        phase=-chopper.phase,
        axle_position=sc.vector(
            value=[0, 0, chopper.distance.value], unit=chopper.distance.unit
        ),
        slit_begin=chopper.open,
        slit_end=chopper.close,
        slit_height=sc.scalar(10.0, unit="cm"),
        radius=sc.scalar(30.0, unit="cm"),
    )
}

workflow[unwrap.RawData] = nxevent_data
workflow[unwrap.Ltotal] = nxevent_data.coords["Ltotal"]
workflow[unwrap.DistanceResolution] = sc.scalar(0.1, unit="m")

workflow.visualize(unwrap.TofData)

#### Unwrapped neutron time-of-arrival

The first step that is computed in the workflow is the unwrapped detector arrival time of each neutron.
This is essentially just `event_time_offset + event_time_zero`.

In [ ]:
da = workflow.compute(unwrap.UnwrappedTimeOfArrival)[
    "detector_number", 2
]  # Look at a single detector
da.bins.concat().value.hist(time_of_arrival=300).plot()

#### Unwrapped neutron time-of-arrival minus pivot time

The next step is to subtract the pivot time to the unwrapped arrival times,
to align the times so that they start at zero.

This allows us to perform a computationally cheap modulo operation on the times below.

In [ ]:
da = workflow.compute(unwrap.UnwrappedTimeOfArrivalMinusStartTime)["detector_number", 2]
f = da.bins.concat().value.hist(time_of_arrival=300).plot()
for i in range(source.pulses):
    f.ax.axvline(
        i * (1.0 / source.frequency).to(unit="us").value, color="k", ls="dotted"
    )
f

The vertical dotted lines here represent the frame period.

#### Unwrapped neutron time-of-arrival modulo the frame period

We now wrap the arrival times with the frame period to obtain well formed (unbroken) set of events.

We also re-add the pivot time offset we had subtracted earlier (to enable to modulo operation).

In [ ]:
da = workflow.compute(unwrap.FrameFoldedTimeOfArrival)["detector_number", 2]
da.bins.concat().value.hist(time_of_arrival=200).plot()

#### Create a lookup table

The chopper information is next used to construct a lookup table that provides an estimate of the real time-of-flight as a function of time-of-arrival.

The `tof` module can be used to propagate a pulse of neutrons through the chopper system to the detectors,
and predict the most likely neutron wavelength for a given time-of-arrival.

We typically have hundreds of thousands of pixels in an instrument,
but it is actually not necessary to propagate the neutrons to 10<sup>5</sup> detectors.

Instead, we make a table that spans the entire range of distances of all the pixels,
with a modest resolution,
and use a linear interpolation for values that lie between the points in the table.

To create the table, we thus:

- run a simulation where a pulse of neutrons passes through the choppers and reaches the sample (or any location after the last chopper)
- propagate the neutrons from the sample to a range of distances that span the minimum and maximum pixel distance from the sample (assuming neutron wavelengths do not change)
- bin the neutrons in both distance and time-of-arrival (yielding a 2D binned data array)
- compute the (weighted) mean wavelength inside each bin
- convert the wavelengths to a real time-of-flight to give our final lookup table

The table can be visualized here:

In [ ]:
table = workflow.compute(unwrap.TimeOfFlightLookupTable)
table.plot()

#### Computing time-of-flight from the lookup

We now use the above table to perform a bilinear interpolation and compute the time-of-flight of every neutron.

In [ ]:
tofs = workflow.compute(unwrap.TofData)

tof_hist = tofs.bins.concat("pulse").hist(tof=sc.scalar(500.0, unit="us"))
pp.plot({det.name: tof_hist["detector_number", i] for i, det in enumerate(detectors)})

### Converting to wavelength

The time-of-flight of a neutron is commonly used as the fundamental quantity from which one can compute the neutron energy or wavelength.

Here, we compute the wavelengths from the time-of-flight using Scippneutron's `transform_coord` utility,
and compare our computed wavelengths to the true wavelengths which are known for the simulated neutrons.

In [ ]:
from scippneutron.conversion.graph.beamline import beamline
from scippneutron.conversion.graph.tof import elastic

# Perform coordinate transformation
graph = {**beamline(scatter=False), **elastic("tof")}

# Define wavelength bin edges
bins = sc.linspace("wavelength", 6.0, 9.0, 101, unit="angstrom")

# Compute wavelengths
wav_hist = (
    tofs.transform_coords("wavelength", graph=graph)
    .bins.concat("pulse")
    .hist(wavelength=bins)
)
wavs = {det.name: wav_hist["detector_number", i] for i, det in enumerate(detectors)}

ground_truth = results["detector"].data.flatten(to="event")
ground_truth = ground_truth[~ground_truth.masks["blocked_by_others"]].hist(
    wavelength=bins
)

wavs["true"] = ground_truth
pp.plot(wavs)

We see that all detectors agree on the wavelength spectrum,
which is also in very good agreement with the true neutron wavelengths.

## Pulse-skipping mode

In some beamline configurations, one wishes to study a wide range of wavelengths at a high flux.
This usually means that the spread of arrival times will spill-over into the next pulse if the detector is placed far enough to yield a good wavelength resolution.

To avoid the next pulse polluting the data from the current pulse,
it is common practice to use a pulse-skipping chopper which basically blocks all neutrons every other pulse.
This could also be every 3 or 4 pulses for very long instruments.

The time-distance diagram may look something like:

In [ ]:
source = tof_pkg.Source(facility="ess", pulses=4)
choppers = [
    tof_pkg.Chopper(
        frequency=14.0 * Hz,
        open=sc.array(dims=["cutout"], values=[0.0], unit="deg"),
        close=sc.array(dims=["cutout"], values=[33.0], unit="deg"),
        phase=35.0 * deg,
        distance=8.0 * meter,
        name="chopper",
    ),
    tof_pkg.Chopper(
        frequency=7.0 * Hz,
        open=sc.array(dims=["cutout"], values=[0.0], unit="deg"),
        close=sc.array(dims=["cutout"], values=[120.0], unit="deg"),
        phase=10.0 * deg,
        distance=15.0 * meter,
        name="pulse-skipping",
    ),
]
detectors = [
    tof_pkg.Detector(distance=60.0 * meter, name="monitor"),
    tof_pkg.Detector(distance=100.0 * meter, name="detector"),
]

model = tof_pkg.Model(source=source, choppers=choppers, detectors=detectors)
results = model.run()
results.plot(cmap="viridis_r", blocked_rays=5000)

### Computing time-of-flight

To compute the time-of-flight in pulse skipping mode,
we can use the same workflow as before.

The only difference is that we set the `PulseStride` to 2 to skip every other pulse.

In [ ]:
workflow = sl.Pipeline(unwrap.providers(), params=unwrap.params())

workflow[unwrap.Facility] = "ess"
workflow[unwrap.PulseStride] = 2
workflow[unwrap.Choppers] = {
    ch.name: DiskChopper(
        frequency=-ch.frequency,
        beam_position=sc.scalar(0.0, unit="deg"),
        phase=-ch.phase,
        axle_position=sc.vector(
            value=[0, 0, ch.distance.value], unit=chopper.distance.unit
        ),
        slit_begin=ch.open,
        slit_end=ch.close,
        slit_height=sc.scalar(10.0, unit="cm"),
        radius=sc.scalar(30.0, unit="cm"),
    )
    for ch in choppers
}

nxevent_data = results.to_nxevent_data()
workflow[unwrap.RawData] = nxevent_data
workflow[unwrap.Ltotal] = nxevent_data.coords["Ltotal"]
workflow[unwrap.DistanceResolution] = sc.scalar(0.5, unit="m")

If we inspect the time-of-flight lookup table,
we can see that the time-of-arrival (toa) dimension now spans longer than the pulse period of 71 ms.

In [ ]:
table = workflow.compute(unwrap.TimeOfFlightLookupTable)
table.plot()

The time-of-flight profiles are then:

In [ ]:
tofs = workflow.compute(unwrap.TofData)

tof_hist = tofs.bins.concat("pulse").hist(tof=sc.scalar(500.0, unit="us"))
pp.plot({det.name: tof_hist["detector_number", i] for i, det in enumerate(detectors)})

### Conversion to wavelength

We now use the `transform_coords` as above to convert to wavelength.

In [ ]:
# Define wavelength bin edges
bins = sc.linspace("wavelength", 1.0, 8.0, 401, unit="angstrom")

# Compute wavelengths
wav_hist = (
    tofs.transform_coords("wavelength", graph=graph)
    .bins.concat("pulse")
    .hist(wavelength=bins)
)
wavs = {det.name: wav_hist["detector_number", i] for i, det in enumerate(detectors)}

ground_truth = results["detector"].data.flatten(to="event")
ground_truth = ground_truth[~ground_truth.masks["blocked_by_others"]].hist(
    wavelength=bins
)

wavs["true"] = ground_truth
pp.plot(wavs)

## Wavelength-frame multiplication mode

The case of wavelength-frame multiplication is treated in exactly the same way as above.

The only difference is that the choppers have multiple openings,
and that the lookup tables have more than one distinct region (subframes) per detector.

But the workflow setup and execution remains exactly the same.

In [ ]:
source = tof_pkg.Source(facility="ess", pulses=2)

slit_width = 3.0
open_edge = sc.linspace("cutout", 0.0, 75, 6, unit="deg")
wfm = tof_pkg.Chopper(
    frequency=14.0 * Hz,
    open=open_edge,
    close=open_edge + slit_width * deg,
    phase=45.0 * deg,
    distance=8.0 * meter,
    name="WFM",
)

slit_width = 25.0
open_edge = sc.linspace("cutout", 0.0, 190, 6, unit="deg")
foc = tof_pkg.Chopper(
    frequency=14.0 * Hz,
    open=open_edge,
    close=open_edge + slit_width * deg,
    phase=85.0 * deg,
    distance=20.0 * meter,
    name="FOC",
)

choppers = [wfm, foc]
detectors = [
    tof_pkg.Detector(distance=30.0 * meter, name="detector"),
]

model = tof_pkg.Model(source=source, choppers=choppers, detectors=detectors)
results = model.run()
results.plot(cmap="viridis_r", blocked_rays=5000)

The signal of the raw `event_time_offset` at the detector wras around the 71 ms mark:

In [ ]:
results.to_nxevent_data().bins.concat().hist(event_time_offset=300).plot()

### Setting up the workflow

We set up the workflow as before, with `PulseStride=1`.

We have two choppers: a WFM chopper with 6 openings,
and a frame-overlap chopper which ensures that the individual subframes do not overlap when they reach the detector.

In [ ]:
workflow = sl.Pipeline(unwrap.providers(), params=unwrap.params())

workflow[unwrap.Facility] = "ess"
workflow[unwrap.Choppers] = {
    ch.name: DiskChopper(
        frequency=-ch.frequency,
        beam_position=sc.scalar(0.0, unit="deg"),
        phase=-ch.phase,
        axle_position=sc.vector(
            value=[0, 0, ch.distance.value], unit=chopper.distance.unit
        ),
        slit_begin=ch.open,
        slit_end=ch.close,
        slit_height=sc.scalar(10.0, unit="cm"),
        radius=sc.scalar(30.0, unit="cm"),
    )
    for ch in choppers
}

nxevent_data = results.to_nxevent_data()
workflow[unwrap.RawData] = nxevent_data
workflow[unwrap.Ltotal] = nxevent_data.coords["Ltotal"]
workflow[unwrap.DistanceResolution] = sc.scalar(0.5, unit="m")

This time, the lookup table has 6 distinct bands separated by empty space,
corresponding to the 6 WFM frames from the 6 chopper openings.

In [ ]:
table = workflow.compute(unwrap.TimeOfFlightLookupTable)
table.plot()

### Computing time-of-flight

The time-of-flight profile resembles the `event_time_offset` profile above,
without the wrapping around at 71 ms.

In [ ]:
tofs = workflow.compute(unwrap.TofData)
pp.plot(tofs.bins.concat().hist(tof=300))

### Conversion to wavelength

Finally, the computed wavelengths show a very good agreement with the true wavelengths:

In [ ]:
# Define wavelength bin edges
bins = sc.linspace("wavelength", 2.0, 12.0, 401, unit="angstrom")

wavs = (
    tofs.transform_coords("wavelength", graph=graph).bins.concat().hist(wavelength=bins)
)

ground_truth = results["detector"].data.flatten(to="event")
ground_truth = ground_truth[~ground_truth.masks["blocked_by_others"]].hist(
    wavelength=bins
)

pp.plot({"wfm": wavs, "true": ground_truth})